In [2]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import seaborn as sns 
import statsmodels.stats.proportion as proportion
from scipy.stats import ttest_ind,mannwhitneyu,shapiro,norm
from statsmodels.stats.weightstats import ztest
from tqdm import tqdm
import timeit
from scipy import stats
import math
from datetime import date, datetime, timedelta
import time
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import warnings
warnings.filterwarnings("ignore")
import clickhouse_connect  



from credential import postgres_secret,clickhouse_dwh_secret

def get_engine(user):
    if user == postgres_secret['user']:
        db_name = postgres_secret['db_name']
        password = postgres_secret['password']
        host = postgres_secret['host']
        engine = create_engine(f'postgresql://{user}:{password}@{host}:6432/{db_name}')
    elif user == clickhouse_dwh_secret['user']:
            db_name = clickhouse_dwh_secret['db_name'] 
            password = clickhouse_dwh_secret['password']
            host = clickhouse_dwh_secret['host']
            engine = create_engine(f'clickhouse://{user}:{password}@{host}:8123/{db_name}')
    return engine
    
connection_clickhouse = clickhouse_connect.get_client(
    host = clickhouse_dwh_secret['host'],
    port= '8123',
    username = clickhouse_dwh_secret['user'],
    password = clickhouse_dwh_secret['password'],
    database='datamarts'
    )

    
def execute(SQL, user):
    start_time = time.time()  # запоминаем время начала выполнения функции
    engine = get_engine(user)
    Session = sessionmaker(bind=engine)  # sessions factory ()
    with Session() as session: # open session
        result = session.execute(text(SQL))
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
        
    end_time = time.time()  # запоминаем время окончания выполнения функции
    execution_time = round(end_time - start_time,4) # вычисляем время выполнения   
    
    print(f"Время выполнения функции: {execution_time} секунд")
    print()
    return df

In [4]:
query = '''CREATE TABLE sandbox.mekhtiev_dau_local ON CLUSTER 'viasat_cluster'
            (
    `date` Date,
    `visitor_id` UUID,
    `profile_id` UUID,
    `reg_date` Date,
    `created_date` Date,
    `first_prolong_date` Date,
    `device_type` String,
    `os_version` String,
    `country` String,
    `bonus_title` String,
    `source` String,
    `client_type` String,
    `free_day` String,
    `user_type` String


             )
             ENGINE = ReplicatedMergeTree('/clickhouse/tables/{shard}/mekhtiev_dau_local', '{replica}')
             ORDER BY date
             '''
execute(query,user = 'kmekhtiev')

Время выполнения функции: 0.5423 секунд



,host,port,status,error,num_hosts_remaining,num_hosts_active
0,dwh03,9000,0,,3,0
1,dwh02,9000,0,,2,0
2,dwh01,9000,0,,1,0
3,dwh04,9000,0,,0,0


In [6]:
query = '''CREATE TABLE sandbox.mekhtiev_dau ON CLUSTER 'viasat_cluster'
            (
    `date` Date,
    `visitor_id` UUID,
    `profile_id` UUID,
    `reg_date` Date,
    `created_date` Date,
    `first_prolong_date` Date,
    `device_type` String,
    `os_version` String,
    `country` String,
    `bonus_title` String,
    `source` String,
    `client_type` String,
    `free_day` String,
    `user_type` String


             )
             ENGINE = Distributed(viasat_cluster, sandbox, mekhtiev_dau_local, rand())
             '''
execute(query,user = 'kmekhtiev')

Время выполнения функции: 0.3592 секунд



,host,port,status,error,num_hosts_remaining,num_hosts_active
0,dwh02,9000,0,,3,0
1,dwh03,9000,0,,2,0
2,dwh01,9000,0,,1,0
3,dwh04,9000,0,,0,0


In [14]:
list_date = pd.date_range("2024-11-01", "2025-02-05", freq='D')
result = []
for date in list_date:
    date_str = date.strftime('%Y-%m-%d')
    query_delete = f" DELETE FROM sandbox.mekhtiev_dau_local ON CLUSTER 'viasat_cluster' WHERE date = '{date}'::date"
    execute(query_delete,user = 'kmekhtiev')
    print(f" Дата {date_str} удалена из таблицы")
    query = f''' INSERT INTO sandbox.mekhtiev_dau

SELECT
    date,
    visitor_id,
    profile_id,
    reg_date,
    created_date,
    first_prolong_date,
    device_type,
    os_version,
    country,
    bonus_title,
    source,
    client_type,
    free_days,
    CASE
        WHEN first_prolong_date!='1970-01-01' AND first_prolong_date<=date THEN 'subs'
        WHEN created_date!='1970-01-01' AND created_date<=date THEN 'trial'
        WHEN reg_date!='1970-01-01' AND reg_date<=date THEN 'reg'
        ELSE 'visitor'
    END AS user_type
FROM
    (
        SELECT
            date,
            visitor_id,
            profile_id,
            bonus_title,
            device_type,
            os_version,
            source,
            country,
            client_type,
            free_days,
            profile_id,
            reg_date,
            promo_type,
            created_at::date AS created_date,
            first_prolong_date::date AS first_prolong_date
        FROM
            datamarts.clean_event
        WHERE
            date='$date'::date 
            AND client_type !='backend'
        GROUP BY
            1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
    )


'''
    
    
    df_temp = execute(query,user = 'kmekhtiev')   
    result.append(df_temp)
    print(f"""Дата '{date_str}' загружена""")
    print()

Время выполнения функции: 0.3769 секунд

 Дата 2024-11-01 удалена из таблицы
Время выполнения функции: 0.7677 секунд

Дата '2024-11-01' загружена

Время выполнения функции: 0.3612 секунд

 Дата 2024-11-02 удалена из таблицы
Время выполнения функции: 0.7825 секунд

Дата '2024-11-02' загружена

Время выполнения функции: 0.694 секунд

 Дата 2024-11-03 удалена из таблицы
Время выполнения функции: 1.3436 секунд

Дата '2024-11-03' загружена

Время выполнения функции: 0.4406 секунд

 Дата 2024-11-04 удалена из таблицы
Время выполнения функции: 0.7373 секунд

Дата '2024-11-04' загружена

Время выполнения функции: 0.3932 секунд

 Дата 2024-11-05 удалена из таблицы
Время выполнения функции: 0.8206 секунд

Дата '2024-11-05' загружена

Время выполнения функции: 0.41 секунд

 Дата 2024-11-06 удалена из таблицы
Время выполнения функции: 0.8131 секунд

Дата '2024-11-06' загружена

Время выполнения функции: 0.9006 секунд

 Дата 2024-11-07 удалена из таблицы
Время выполнения функции: 1.4115 секунд

Дат